# Подготовка к работе

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import ast
import re
from pathlib import Path
import pickle
import gc

from sklearn.preprocessing import MultiLabelBinarizer, OrdinalEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import (
	make_scorer,
    jaccard_score,
    f1_score,
    accuracy_score,
)

import warnings
warnings.filterwarnings('ignore')

import random
random.seed(42)
np.random.seed(42)

# Предобработка

Качаем датасет:

In [2]:
df = pd.read_csv(Path("..") / "data" / "habr.csv", on_bad_lines='skip').sample(100000) # 100000, чтобы не сильно память ело

Вспомним, что там было:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 2964 to 105027
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            100000 non-null  int64  
 1   text          99764 non-null   object 
 2   keywords      100000 non-null  object 
 3   hubs          100000 non-null  object 
 4   username      99707 non-null   object 
 5   reading_time  99997 non-null   float64
 6   title         100000 non-null  object 
 7   time          100000 non-null  object 
 8   status        100000 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 7.6+ MB


In [4]:
df.sample(7)

,id,text,keywords,hubs,username,reading_time,title,time,status
117620,708318,Находит меня хеадхантерша из другой страны и п...,"['собеседование', 'c/c++ программирование']",['Программирование'],fronda,3.0,Стрёмное собеседование в Яндекс / Хабр,2022-12-28 18:21:59,ok
18932,434044,Последний месяц все кому не лень пишут что 2FA...,"['IAM', 'identity management', 'identity acces...","['Информационная безопасность', 'Серверное адм...",apcsb,5.0,Двухфакторная аутентификация (2FA) устойчивая ...,2018-12-22 21:36:05,ok
117237,707448,"Пользователирассказали, как они используют Cha...","['tinder', 'chatgpt', 'соцсети', 'знакомства',...","['Алгоритмы', 'Машинное обучение', 'Искусствен...",maybe_elf,1.0,Посетители Tinder начали использовать ChatGPT ...,2022-12-24 07:03:46,ok
45922,498174,"Когда вы разрабатываете продукт, каждая новая ...","['а/б тестирование', 'a/b-тестирование', 'спли...","['Блог компании Badoo', 'Повышение конверсии',...",kislovm,5.0,Как перестать беспокоиться и начать верить A/B...,2020-04-21 12:00:06,ok
104896,680062,Два сверхмалых искусственных спутника-исследов...,"['кубсаты', 'сверхмалые спутники', 'спутники',...","['Научно-популярное', 'Космонавтика', 'IT-комп...",IgnatChuker,2.0,В августе на орбиты должны отправить два сверх...,2022-07-30 23:04:51,ok
148384,781124,Кому нужны книжки без картинок … или хоть стиш...,"['bpmn', 'epc', 'flowcharts', 'workflow', 'gra...","['Анализ и проектирование систем', 'Алгоритмы'...",itGuevara,24.0,WF2M сеть. Формализм и математика workflow / Хабр,2023-12-17 11:14:02,ok
131385,739642,Новинка от Onyx Boox — это компактная электрон...,"['onyx boox', 'Обзор', 'Обзор электронной книг...",['Гаджеты'],Lexus08,6.0,Обзор электронной книги Onyx Boox Galileo c эк...,2023-06-04 18:16:52,ok


In [5]:
df.describe(include="all")

,id,text,keywords,hubs,username,reading_time,title,time,status
count,100000.000000,99764,100000,100000,99707,99997.000000,100000,100000,100000
unique,NaN,99752,94615,54558,19793,NaN,99787,99861,1
top,NaN,Deleted,['черт знает что'],['Информационная безопасность'],denis-19,NaN,ТОП-3 ИБ-событий недели по версии Jet CSIRT / ...,2019-10-02 07:00:02,ok
freq,NaN,3,110,508,5070,NaN,74,4,100000
mean,586076.997670,NaN,NaN,NaN,NaN,5.965179,NaN,NaN,NaN
std,121242.082401,NaN,NaN,NaN,NaN,5.764434,NaN,NaN,NaN
min,390007.000000,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN
25%,479563.500000,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN
50%,568918.000000,NaN,NaN,NaN,NaN,4.000000,NaN,NaN,NaN
75%,700402.000000,NaN,NaN,NaN,NaN,8.000000,NaN,NaN,NaN


# Обучение

In [6]:
class HabrPreprocessor(BaseEstimator, TransformerMixin):

    def __init__(self, min_hub_freq=500):
        self.min_hub_freq = min_hub_freq
        self.hub_filter_ = None

    @classmethod
    def parse_list(cls, x):
        if isinstance(x, list):
            return x
        if not isinstance(x, str):
            return []
        parsed = ast.literal_eval(x)
        if isinstance(parsed, list):
            return [str(i) for i in parsed]
        return []
    
    def __cast_types(self, X):
        X["text"] = X["text"].astype("string")
        X["title"] = X["title"].astype("string")
        X["username"] = X["username"].astype("string")
        X["reading_time"] = X["reading_time"].astype("uint8")
        X["timestamp"] = pd.to_datetime(X["time"]).astype("int64") // 10**9
        return X

    def __prepare_na(self, X):
        X["username"] = X["username"].fillna("")
        X["reading_time"] = X["reading_time"].fillna(0)
        X["username"].fillna(X["username"].mode()[0], inplace=True)
        X["reading_time"].fillna(X["reading_time"].median(), inplace=True)
        return X

    def __drop(self, X):
        columns = ["status", "time"]
        return X.drop(columns=columns)
    
    def fit(self, X, y=None):
        if X.get("hubs"):
            all_hubs = []
            for hubs_str in X["hubs"]:
                hubs = self.parse_list(hubs_str)
                all_hubs.extend(hubs)
            
            hub_counts = Counter(all_hubs)
            self.hub_filter_ = {h for h, c in hub_counts.items() if c >= self.min_hub_freq}
        return self
    
    def __filter_hubs(self, hubs):
        if self.hub_filter_ is None:
            raise ValueError("fit() не вызван")
        return [h for h in hubs if h in self.hub_filter_]

    def transform(self, X):
        X_ = X.copy()
        X_ = self.__prepare_na(X_)
        X_ = self.__cast_types(X_)
        
        X_["keywords"] = X_["keywords"].apply(
            lambda x: " ".join(set(self.parse_list(x)))
        )
        if X_.get("hubs"):
            X_["hubs"] = X_["hubs"].apply(
                lambda hubs: self.__filter_hubs(list(hubs))
            )
            X_["hubs"] = X_["hubs"].apply(
            lambda x: set([
                h for h in self.parse_list(x)
                if not h.lower().startswith("блог компании")
            ])
        )
        X_ = self.__drop(X_)

        return X_.reset_index(drop=True)

    @classmethod
    def clean_text_by_minimax(cls, X_, min_=1000, max_=200000):
        text_lens = X_["text"].apply(lambda x: len(str(x)))
        return X_[(min_ < text_lens) & (text_lens < max_)]


In [7]:
preprocessor = HabrPreprocessor(min_hub_freq=350)

train_df = df.sample(5000)
train_df.dropna(subset=['text'], inplace=True)
train_df.dropna(subset=['hubs'], inplace=True)
train_df = preprocessor.clean_text_by_minimax(train_df)
train_df

,id,text,keywords,hubs,username,reading_time,title,time,status
64514,542140,"Clubhouse — новая социальная сеть, которая вз...","['Clubhouse', 'мессенджер', 'социальные сети',...","['Мессенджеры', 'Социальные сети', 'Управление...",IntellaRus,5.0,Новая парадигма рекрутинга: кого и как искать ...,2021-02-12 13:27:50,ok
31278,463431,Во время работы и прогулок я часто слушаю фоно...,"['internet radio', 'android', 'C++', 'sqlite3'...","['Android', 'Софт']",ababo,3.0,"Умное музыкальное радио, не требующее постоянн...",2019-08-12 22:06:38,ok
31731,464531,20 августа 2019 года компания Samsung открылаф...,"['Samsung', 'Tmall', 'AliExpress']","['IT-компании', 'Гаджеты', 'Смартфоны', 'Управ...",alizar,1.0,Samsung открыл фирменный магазин на Tmall для ...,2019-08-21 11:09:29,ok
134189,746430,"Люди — существа социальные. Те, кто чувствует ...","['фмрт', 'одиночество']","['Научно-популярное', 'Мозг', 'Здоровье']",SLY_G,3.0,Исследование: одинокие люди воспринимают мир н...,2023-07-06 19:01:01,ok
45894,498118,Многие читатели Хабра уже знают и любят наши I...,"['конференции', 'Heisenbug', 'DotNext', 'Mobiu...","['Блог компании JUG Ru Group', 'Конференции']",phillennium,7.0,Конференции на удалёнке: онлайн-трансформация ...,2020-04-20 15:17:52,ok
...,...,...,...,...,...,...,...,...,...
51621,511508,"Автор — сэр Тим Бернерс-Ли, изобретатель URI, ...","['URL', 'URI', 'крутость']","['Доменные имена', 'Веб-дизайн', 'Веб-разработ...",m1rko,10.0,Крутые URI не изменяются / Хабр,2020-07-18 05:58:04,ok
75454,568012,Подключение нейросети для автоматизации любого...,"['М.Видео', 'Эльдорадо', 'чат-боты', 'технолог...","['Блог компании М.Видео-Эльдорадо', 'Искусстве...",mvideo,6.0,"М.Помощник: привязали нейросеть к чат-боту, чт...",2021-07-29 08:19:12,ok
116701,706296,Google добавил в Gmail сквозное шифрование (E2...,"['google', 'gmail', 'безопасность', 'информаци...","['Блог компании Cloud4Y', 'Информационная безо...",Cloud4Y,2.0,В Gmail появилось сквозное шифрование / Хабр,2022-12-19 09:26:04,ok
95773,659677,Свой первый отчёт с данными о работе процесса ...,"['agile', 'scrum', 'kanban', 'lean', 'devops',...","['Agile', 'DevOps']",Cleverics,2.0,Метрики потока создания ценности / Хабр,2022-04-07 15:19:44,ok


In [ ]:
X = train_df
y_raw = train_df['hubs']
X = train_df.drop(columns=['hubs'])

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y_raw)

ct = ColumnTransformer(
    transformers=[
        ("tfidf_text", TfidfVectorizer(max_features=5000), "text"),
        ("tfidf_keywords", TfidfVectorizer(max_features=2000), "keywords"),
        ("tfidf_title", TfidfVectorizer(max_features=100), "title"),
        ("username_enc", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), ["username"]),
        ("reading_time_scale", StandardScaler(), ["reading_time"]),
        ("timestamp_scale", StandardScaler(), ["timestamp"]),
    ],
    remainder="drop"
)

model = Pipeline([
    ("habr_pre", preprocessor),
    ("features", ct),
    ("pca", TruncatedSVD(n_components=100, algorithm="randomized", n_iter=5)),
    ("knn", OneVsRestClassifier(KNeighborsClassifier(
            n_jobs=-1,
            
            algorithm='ball_tree',
			leaf_size=800,
        ), n_jobs=-1))
])

param_grid = {
    "knn__estimator__n_neighbors": [3, 5, 7, 9],
}

jaccard_scorer = make_scorer(jaccard_score, average='micro')
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=3,
    scoring=jaccard_scorer,
    n_jobs=-1,
	verbose=3,
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV 2/3] END .....knn__estimator__n_neighbors=3;, score=0.534 total time=  48.9s
[CV 1/3] END .....knn__estimator__n_neighbors=3;, score=0.500 total time=  50.5s
[CV 1/3] END .....knn__estimator__n_neighbors=5;, score=0.532 total time=  51.2s
[CV 3/3] END .....knn__estimator__n_neighbors=3;, score=0.537 total time=  53.1s
[CV 2/3] END .....knn__estimator__n_neighbors=5;, score=0.532 total time=  53.3s
[CV 3/3] END .....knn__estimator__n_neighbors=5;, score=0.545 total time=  55.1s
[CV 2/3] END .....knn__estimator__n_neighbors=7;, score=0.539 total time=  55.4s
[CV 1/3] END .....knn__estimator__n_neighbors=7;, score=0.548 total time=  55.8s
[CV 3/3] END .....knn__estimator__n_neighbors=7;, score=0.558 total time=  32.9s
[CV 1/3] END .....knn__estimator__n_neighbors=9;, score=0.545 total time=  32.2s
[CV 2/3] END .....knn__estimator__n_neighbors=9;, score=0.551 total time=  31.7s
[CV 3/3] END .....knn__estimator__n_neighbors=9;,

,estimator,Pipeline(step... n_jobs=-1))])
,param_grid,"{'knn__estimator__n_neighbors': [3, 5, ...]}"
,scoring,make_scorer(j...average=micro)
,n_jobs,-1
,refit,True
,cv,3
,verbose,3
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,min_hub_freq,350


In [10]:

def label_coverage(y_pred):
    """
    Доля уникальных хабов, которые модель предсказала хотя бы один раз.
    y_pred — бинарная матрица shape (n_samples, n_classes)
    """
    return np.sum(y_pred.sum(axis=0) > 0) / y_pred.shape[1]

def print_metrics(X_test, y_test, model):
    y_pred = model.predict(X_test)
    jaccard_micro = jaccard_score(y_test, y_pred, average='micro')
    jaccard_macro = jaccard_score(y_test, y_pred, average='macro')
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    acc = accuracy_score(y_test, y_pred)
    coverage = label_coverage(y_pred)

    print("===== Итоговые метрики модели =====")
    print(f"Label Coverage Rate: {coverage:.4f}")
    print(f"Jaccard (micro): {jaccard_micro:.4f}")
    print(f"Jaccard (macro): {jaccard_macro:.4f}")
    print(f"F1 (micro):      {f1_micro:.4f}")
    print(f"F1 (macro):      {f1_macro:.4f}")
    print(f"Accuracy:         {acc:.4f}")

print_metrics(X_test, y_test, grid)
print("---------------------------------------")
print(f"Лучший скор на кросс-валидации (GridSearch): {grid.best_score_:.4f}")
print(f"Лучшие параметры: {grid.best_params_}")

===== Итоговые метрики модели =====
Label Coverage Rate: 0.7770
Jaccard (micro): 0.5426
Jaccard (macro): 0.1999
F1 (micro):      0.7035
F1 (macro):      0.2680
Accuracy:         0.0082
---------------------------------------
Лучший скор на кросс-валидации (GridSearch): 0.5511
Лучшие параметры: {'knn__estimator__n_neighbors': 9}


# Увеличим выборку

In [11]:
train_df_result = df.sample(30000)
train_df_result.dropna(subset=['text'], inplace=True)
train_df_result.dropna(subset=['hubs'], inplace=True)
train_df_result = preprocessor.clean_text_by_minimax(train_df_result)
train_df_result

,id,text,keywords,hubs,username,reading_time,title,time,status
92503,652079,"Поданным""Медиалогии"", спустя год после вступле...","['Мат', 'брань', 'запрет', 'соцсети']","['IT-компании', 'Социальные сети', 'Статистика...",denis-19,1.0,Спустя год после запрета мата в соцсетях ругат...,2022-02-17 10:00:17,ok
31448,463835,Сетевые технологии сделали мир совсем маленьки...,"['iot', 'промышленность', 'интернет вещей', 'о...","['Блог компании ITT Solutions', 'Интернет веще...",Zyxel_CIS,4.0,Этот опасный IoT: угрозы бизнесу и способы реш...,2019-08-15 16:29:20,ok
148718,781936,"Туннель IPIP, как можно понять из его названия...","['Маршрутизация', 'ipv4', 'подсети', 'роутинг'...",['Настройка Linux'],DmitryITLin,2.0,Маршрутизация подсети IPv4 через IPIP / Хабр,2023-12-20 11:37:15,ok
23596,445212,В данной статье пойдёт краткое повествование о...,"['netbios', 'unix', 'windows', 'enumeration', ...","['*nix', 'IT-инфраструктура', 'Информационная ...",USSCLTD,6.0,NetBIOS в руках хакера / Хабр,2019-03-25 11:46:35,ok
70919,557254,Через несколько дней начнется чемпионат Европы...,"['футбол', 'тотализатор', 'ставки на спорт']","['Блог компании lsFusion', 'Игры и игровые кон...",CrushBy,8.0,Организуем собственный мини-тотализатор на мат...,2021-06-07 07:06:15,ok
...,...,...,...,...,...,...,...,...,...
6262,404049,Предисловие к первой частиМоделирование паровы...,"['пример', 'matlab', 'паровая турбина']","['Научно-популярное', 'Энергия и элементы пита...",mbureau,11.0,Линеаризованная расходная характеристика паров...,2017-05-22 18:01:42,ok
16542,428347,Некоторые люди как-то неправильно поняли WebAs...,['webassembly'],"['Блог компании Инфопульс Украина', 'Firefox',...",tangro,18.0,Будущее WebAssembly в виде «дерева навыков» / ...,2018-10-31 11:06:19,ok
53181,515204,"Друзья, всем привет! Меня зовут Коля Архипов, ...","['dctech', 'delivery club', 'foodtech', 'resea...","['Блог компании Delivery Club Tech', 'Блог ком...",nikkiola,15.0,Как мы побеждаем неопределенность в Delivery C...,2020-08-18 11:44:06,ok
64131,541214,"Первая частьмаленького ""срывания покрова"" о ра...","['linux', 'swap', 'оптимизация', 'тестирование']","['Настройка Linux', 'Linux', 'Серверное админи...",outlingo,7.0,"Почему линукс использует swap-файл, часть 2 / ...",2021-02-07 12:16:53,ok


In [12]:
X_result = train_df_result
y_raw_result = train_df_result['hubs']
X_result = train_df_result.drop(columns=['hubs'])

mlb_result = MultiLabelBinarizer()
y_result = mlb_result.fit_transform(y_raw_result)

n_neighbors = 3

X_train_result, X_test_result, y_train_result, y_test_result = train_test_split(X_result, y_result, test_size=0.2)

model_result = Pipeline([
    ("habr_pre", preprocessor),
    ("features", ct),
    ("svd", TruncatedSVD(n_components=100, algorithm="randomized", n_iter=5)),
    ("knn", OneVsRestClassifier(KNeighborsClassifier(
        n_neighbors=n_neighbors,
        n_jobs=-1,
        algorithm='ball_tree',
        leaf_size=800
    ), n_jobs=-1))
])

In [13]:
model_result.fit(X_train_result, y_train_result)

,steps,"[('habr_pre', ...), ('features', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,min_hub_freq,350
,transformers,"[('tfidf_text', ...), ('tfidf_keywords', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False


In [14]:
print_metrics(X_test_result, y_test_result, model_result)

===== Итоговые метрики модели =====
Label Coverage Rate: 0.9116
Jaccard (micro): 0.6076
Jaccard (macro): 0.3796
F1 (micro):      0.7559
F1 (macro):      0.5042
Accuracy:         0.0648


In [17]:
with open("model.pkl", "wb") as f:
    pickle.dump(model_result, f)